# **Real-Time Object Detection using OpenCV and DNN**


### Author : Asma Dammak

This project demonstrates a simple implementation of real-time object detection using OpenCV's DNN module. It utilizes a pre-trained SSD MobileNet v3 model for object detection from live webcam feed. The detections are optimized by applying Non-Maximum Suppression (NMS) to filter out overlapping bounding boxes.

### **Features**
Real-time Object Detection: Detects objects in real-time using your webcam.
SSD MobileNet v3: A fast and lightweight object detection model.
Non-Maximum Suppression (NMS): Eliminates redundant bounding boxes to ensure accurate detections.
COCO Dataset Classes: Detects objects from the COCO dataset (80 classes).

### **Requirements**
Python 3.x

OpenCV 4.x

NumPy

In [1]:
#import libraries 
import cv2
import numpy as np

# Confidence threshold for object detection
thres = 0.5  
# Non-Maximum Suppression threshold
nms_threshold = 0.3  

# Open the webcam
cap = cv2.VideoCapture(0)


In [2]:
# Load class names
classFile = 'coco.names'
with open(classFile, 'rt') as f:
    classNames = f.read().strip().split('\n')

print(classNames)

['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'hair brush']


In [3]:
# Load the pre-trained model
configPath = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'frozen_inference_graph.pb'
net = cv2.dnn_DetectionModel(weightsPath, configPath)
net.setInputSize(320, 320) 
net.setInputScale(1.0 / 127.5)  
net.setInputMean((127.5, 127.5, 127.5)) 
net.setInputSwapRB(True) 

< cv2.dnn.Model 00000151A27B2130>

In [4]:
# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    success, img = cap.read()  
    if not success:
        print("Failed to capture image")
        break  

    # Perform object detection
    classIds, confs, bbox = net.detect(img, confThreshold=thres)

    if classIds is not None:
        # Convert results to lists
        bbox = list(bbox)
        confs = list(np.array(confs).flatten())
        classIds = list(np.array(classIds).flatten())

        # Apply Non-Maximum Suppression to eliminate overlapping boxes
        indices = cv2.dnn.NMSBoxes(bbox, confs, thres, nms_threshold)

        # Check if indices is not empty
        if len(indices) > 0:
            for i in indices.flatten(): 
                box = bbox[i]
                x, y, w, h = box[0], box[1], box[2], box[3]
                label = classNames[classIds[i] - 1].upper()
                confidence = confs[i]
                cv2.rectangle(img, (x, y), (x + w, y + h), color=(0, 255, 0), thickness=2)
                cv2.putText(img, f'{label} {confidence:.2f}', (x + 10, y + 30),
                            cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

    # Display the image with detections
    cv2.imshow("Output", img)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

KeyboardInterrupt: 

In [5]:
# Release the capture and close all windows
cap.release()
cv2.destroyAllWindows()